In [1]:
import os
import json
import time
import random
import numpy as np
import imgaug as ia
from imgaug import augmenters as iaa
import cv2
import matplotlib.pyplot as plt
import shutil
from PIL import Image
import copy

In [8]:
def imgs_Augmentation(
                    raw_images_path,
                    raw_annotation,
                    new_images_path,
                    new_annotation,
                    Fliplr_ratio = 0.3,
                    Flipud_ratio = 0.3,
                    Noise_ratio = 0.03,
                    weather_ratio = 0.001
                    ):
    assert os.path.exists(raw_annotation),f"{raw_annotation} don't exist !!!"
    assert os.path.exists(raw_images_path),f"{raw_images_path} don't exist !!!"
    with open(raw_annotation) as json_file:
        raw_anns = json.load(json_file)
        imgs_info = raw_anns.keys()
        imgs_num = len(imgs_info)
        #print(f"Total_image: {imgs_num}")
        
        # Fliplr
        print(f"Dealing with Fliplr ....")
        after_fliplr_info = {}
        before_fliplr_keys = []
        before_fliplr_imgpaths = []
        before_fliplr_imglist = []
        before_fliplr_imgwidth = []
        after_fliplr_imgpaths = []
        after_fliplr_imglist = []
        random.seed(111)        
        Fliplr_id = random.sample(range(imgs_num),int(imgs_num*Fliplr_ratio))
        #Fliplr_id.sort()
        #print(f"0.Fliplr_iamges: {len(Fliplr_id)}")
        i=0
        for i, key in enumerate(imgs_info):
            if i in Fliplr_id:
                before_fliplr_keys.append(key)
                rawimg_name = raw_anns[key]['filename']
                rawimg_path  = os.path.join(raw_images_path,rawimg_name)
                assert os.path.exists(rawimg_path), f"{rawimg_path} doesn't exist !!!"
                before_fliplr_imgpaths.append(rawimg_path)
        #print(f"1.Fliplr_iamges: {len(before_fliplr_imgpaths)}")
        for img_path in before_fliplr_imgpaths:
            rawimg = plt.imread(img_path)
            Hight,width = rawimg.shape[:2]
            before_fliplr_imgwidth.append(width)
            before_fliplr_imglist.append(rawimg)
        #print(f"before_fliplr_imglist: {len(before_fliplr_imglist)}")
        seq = iaa.Sequential([iaa.Fliplr(1)])
        after_fliplr_imglist = seq.augment_images(before_fliplr_imglist)
        #print(f"after_fliplr_imglist: {len(after_fliplr_imglist)}")
        i = 0
        for i, key in enumerate(before_fliplr_keys):
            #print(key)
            augimg_name = f"{key[0:key.rfind('.jpg')]}_fliplr.jpg"
            augimg_path = os.path.join(new_images_path,augimg_name)
            #print(augimg_path)
            plt.imsave(augimg_path,after_fliplr_imglist[i])
            assert os.path.exists(augimg_path), f"{augimg_path} is not exist!!!"
            after_fliplr_imgpaths.append(augimg_path)
            size = os.path.getsize(augimg_path)
            augimg_size_name = augimg_name+str(size)
            #print(augimg_size_name)
            after_fliplr_info[augimg_size_name] = {}
            after_fliplr_info[augimg_size_name]['filename'] = augimg_name
            after_fliplr_info[augimg_size_name]['size'] = size
            after_fliplr_info[augimg_size_name]['regions'] = \
                copy.deepcopy(raw_anns[key]['regions'])
            after_fliplr_info[augimg_size_name]['file_attributes'] = { }
            width = before_fliplr_imgwidth[i]
            for r in after_fliplr_info[augimg_size_name]['regions']:
                #print(r.keys())
                shape = r['shape_attributes']
                #print(shap.keys())
                Xs = r['shape_attributes']['all_points_x']
                Ys = r['shape_attributes']['all_points_y']
                #print(r['shape_attributes']['all_points_x'])                
                for j in range(len(Xs)):
                    if Xs[j] <= width and Xs[j] >= 0:
                        Xs[j] = width - Xs[j]
                    elif Xs[j] < 0:
                        Xs[j] = width
                    elif Xs[j] > width:
                        Xs[j] = 0
 
        # Flipud
        print(f"Dealing with Flipud ....")
        after_flipud_info = {}
        before_flipud_keys = []
        before_flipud_imgpaths = []
        before_flipud_imglist = []
        befor_flipud_imgheight = []
        after_flipud_imgpaths = []
        after_flipud_imglist = []
        random.seed(333)
        Flipud_id = random.sample(range(imgs_num),int(imgs_num*Flipud_ratio))
        #print(f"0.Flipud_iamges: {len(Flipud_id)}")
        i=0
        for i, key in enumerate(imgs_info):
            if i in Flipud_id:
                before_flipud_keys.append(key)
                rawimg_name = raw_anns[key]['filename']
                rawimg_path  = os.path.join(raw_images_path,rawimg_name)
                assert os.path.exists(rawimg_path), f"{rawimg_path} doesn't exist !!!"
                before_flipud_imgpaths.append(rawimg_path)
        #print(f"1.Flipud_iamges: {len(before_flipud_imgpaths)}")
        for img_path in before_flipud_imgpaths:
            rawimg = plt.imread(img_path)
            Hight,width = rawimg.shape[:2]
            befor_flipud_imgheight.append(Hight)
            before_flipud_imglist.append(rawimg)
        #print(f"before_flipud_imglist: {len(before_flipud_imglist)}")
        seq = iaa.Sequential([iaa.Flipud(1)])
        after_flipud_imglist = seq.augment_images(before_flipud_imglist)
        #print(f"after_flipud_imglist: {len(after_flipud_imglist)}")
        i = 0
        for i, key in enumerate(before_flipud_keys):
            #print(key)
            augimg_name = f"{key[0:key.rfind('.jpg')]}_flipud.jpg"
            augimg_path = os.path.join(new_images_path,augimg_name)
            #print(augimg_path)
            plt.imsave(augimg_path,after_flipud_imglist[i])
            assert os.path.exists(augimg_path), f"{augimg_path} is not exist!!!"
            after_flipud_imgpaths.append(augimg_path)
            size = os.path.getsize(augimg_path)
            augimg_size_name = augimg_name+str(size)
            #print(augimg_size_name)
            after_flipud_info[augimg_size_name] = {}
            after_flipud_info[augimg_size_name]['filename'] = augimg_name
            after_flipud_info[augimg_size_name]['size'] = size
            after_flipud_info[augimg_size_name]['regions'] = \
                copy.deepcopy(raw_anns[key]['regions'])
            after_flipud_info[augimg_size_name]['file_attributes'] = {}
            # deal with regions after image flipe updown
            k = 0
            height = befor_flipud_imgheight[i]
            for r in after_flipud_info[augimg_size_name]['regions']:
                shape = r['shape_attributes']
                Xs = r['shape_attributes']['all_points_x']
                Ys = r['shape_attributes']['all_points_y']
                #print("Before: ",r['shape_attributes']['all_points_y'])                
                for j in range(len(Ys)):
                    if Ys[j] <= height and Ys[j] >= 0:
                        Ys[j] = height - Ys[j]
                    elif Ys[j] < 0:
                        Ys[j] = height
                    elif Ys[j] > height:
                        Ys[j] = 0
                #print("after: ",r['shape_attributes']['all_points_y'])
                #print(Xs)
                #print(raw_anns[key]['regions'][k]['shape_attributes']['all_points_y'])
                k +=1
        

        # weather
        print(f"Dealing with cloud ....")
        after_weather_info = {}
        before_weather_keys = []
        before_weather_imgpaths = []
        before_weather_imglist = []
        after_weather_imgpaths = []
        after_weather_imglist = []
        random.seed(555)
        weather_id = random.sample(range(imgs_num),int(imgs_num*weather_ratio))
        #print(f"0. weather_iamges: {len(weather_id)}")
        i=0
        for i, key in enumerate(imgs_info):
            if i in weather_id:
                before_weather_keys.append(key)
                rawimg_name = raw_anns[key]['filename']
                rawimg_path  = os.path.join(raw_images_path,rawimg_name)
                assert os.path.exists(rawimg_path), f"{rawimg_path} doesn't exist !!!"
                before_weather_imgpaths.append(rawimg_path)
        #print(f"1. weather_iamges: {len(before_weather_imgpaths)}")
        for img_path in before_weather_imgpaths:
            rawimg = plt.imread(img_path)
            before_weather_imglist.append(rawimg)
        #print(f"2. before_weather_imglist: {len(before_weather_imglist)}")
        seq = iaa.Sequential([iaa.Clouds(seed = 1)])
        after_weather_imglist = seq.augment_images(before_weather_imglist)
        #print(f"3. after_weather_imglist: {len(after_weather_imglist)}")
        i = 0
        for i, key in enumerate(before_weather_keys):
            augimg_filename = f"{key[0:key.rfind('.jpg')]}_clouds.jpg"
            augimg_path = os.path.join(new_images_path, augimg_filename)
            
            plt.imsave(augimg_path, after_weather_imglist[i])
            assert os.path.exists(augimg_path), f"{augimg_path} is not exist!!!"
            after_weather_imgpaths.append(augimg_path)
            
            size = os.path.getsize(augimg_path)
            augimg_key = augimg_name + str(size)
            
            after_weather_info[augimg_key] = {}
            after_weather_info[augimg_key]['filename'] = augimg_name
            after_weather_info[augimg_key]['size'] = size
            after_weather_info[augimg_key]['regions'] = copy.deepcopy(raw_anns[key]['regions'])
            after_weather_info[augimg_key]['file_attributes'] = { }
        
        # GaussianNoise
        print(f"Dealing with GaussianNoise ....")
        after_noise_info = {}
        before_noise_keys = []
        before_noise_imgpaths = []
        before_noise_imglist = []
        after_noise_imgpaths = []
        after_noise_imglist = []
        random.seed(777)
        noise_id = random.sample(range(imgs_num),int(imgs_num*Noise_ratio))
        
        for i, key in enumerate(imgs_info):
            if i in noise_id:
                before_noise_keys.append(key)
                rawimg_name = raw_anns[key]['filename']
                rawimg_path  = os.path.join(raw_images_path,rawimg_name)
                assert os.path.exists(rawimg_path), f"{rawimg_path} doesn't exist !!!"
                before_noise_imgpaths.append(rawimg_path)
        #print(f"1. noise_iamges: {len(before_noise_imgpaths)}")
        for img_path in before_noise_imgpaths:
            rawimg = plt.imread(img_path)
            before_noise_imglist.append(rawimg)
        #print(f"2. before_noise_imglist: {len(before_noise_imglist)}")
        seq = iaa.Sequential([iaa.AdditiveGaussianNoise(scale=0.2*255, per_channel=0.5)])
        after_noise_imglist = seq.augment_images(before_noise_imglist)
        #print(f"3. after_noise_imglist: {len(after_noise_imglist)}")
        
        for i, key in enumerate(before_noise_keys):
            augimg_filename = f"{key[0:key.rfind('.jpg')]}_noise.jpg"
            augimg_path = os.path.join(new_images_path, augimg_filename)
            
            plt.imsave(augimg_path, after_noise_imglist[i])
            assert os.path.exists(augimg_path), f"{augimg_path} is not exist!!!"
            after_noise_imgpaths.append(augimg_path)
            
            size = os.path.getsize(augimg_path)
            augimg_key = augimg_filename + str(size)
            
            after_noise_info[augimg_key] = {}
            after_noise_info[augimg_key]['filename'] = augimg_name
            after_noise_info[augimg_key]['size'] = size
            after_noise_info[augimg_key]['regions'] = copy.deepcopy(raw_anns[key]['regions'])
            after_noise_info[augimg_key]['file_attributes'] = {}

        # Copy raw image to new_images_path
        print("Copy raw images to new direction...")
        for i, key in enumerate(imgs_info):
            rawimg_name = raw_anns[key]['filename']
            rawimg_path  = os.path.join(raw_images_path,rawimg_name)
            copyimg_path = os.path.join(new_images_path,rawimg_name)
            assert os.path.exists(rawimg_path), f"{rawimg_path} doesn't exist !!!"
            #print(rawimg_path)
            if not os.path.exists(copyimg_path):
                shutil.copy(rawimg_path,copyimg_path)
            assert os.path.exists(copyimg_path), f"{copyimg_path} doesn't exist !!!"
            #print(copyimg_path)
        print("Generating the annotation file....")
        raw_anns = {
            **raw_anns,
            **after_fliplr_info,
            **after_flipud_info,
            **after_weather_info,
            **after_noise_info
        }
    
        with open(new_annotation, "w") as f:
            json.dump(raw_anns,f)
    
        if os.path.exists(new_annotation):
            print("It may be successful!")
        else:
            print("let us start again !!!") 
    return new_annotation

In [9]:
data_in_path = os.path.join("data", "train")
ann_in_path = os.path.join(data_in_path, "train.json")

data_out_path = os.path.join("data", "train_aug")
ann_out_path = os.path.join(data_out_path, "train_aug.json")

print(f"raw images annotation file: {ann_in_path}")
print(f"new images annotation file: {ann_out_path}")

raw images annotation file: data/train/train.json
new images annotation file: data/train_aug/train.json


In [10]:
imgs_Augmentation(
    data_in_path,
    ann_in_path,
    data_out_path,
    ann_out_path
)

Dealing with Fliplr ....
Dealing with Flipud ....
Dealing with cloud ....
Dealing with GaussianNoise ....
Copy raw images to new direction...
Genration the annotation file....
It may be successful!


'data/train_aug/train.json'